In [ ]:
import os


api_key = os.getenv('MY_API_KEY')
api_base = os.getenv('MY_API_BASE')

llm两种应用范式
* tooled-argumented
工具增强，指的是大模型结合外部工具来完成复杂任务的能力；类似bingapi做天气搜索

* agent
是更上一层的抽象，只作为一个智能体，强调利用大模型作为推理引擎。自主规划，决定以一个什么样的顺序，使用某种工具来完成一个复杂的任务

提示词技术-ReACT框架
https://www.promptingguide.ai/techniques/react

伪代码看agent结合tool怎么工作的
```

Agent{
prompt = """
Answer the following questions as best you can. You have access to the following tools:
{tool}
....
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question"""
"""

run(question){
    tools = load_tools()
    prompt.format('tool',tools)
    add_system(prompt)
    answer = ""
    while(!answer.start_with("Final Answer")){
        answer = llm.predict()
        if(answer.start_with("Action")){
            action,input = output_parser(answer)
            real_action = get_action(action)
            obs = real_action.do_action(input)
            add_system(obs)
        }
    }
}

}

```

看看langchain中怎么使用的，在langchain中tool和agent通常是一起使用的

In [15]:
from langchain.agents import AgentType,load_tools,initialize_agent,AgentExecutor,ZeroShotAgent
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(api_key=api_key,base_url=api_base)

tools = load_tools(["arxiv"])

agent = initialize_agent(tools,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

agent.run("what's the paper 1605.08386 about?")



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

arxiv: A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [arxiv]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: what's the paper 1605.08386 about?
Thought:

> Finished chain.
I should use the a

'The paper 1605.08386 is titled "Heat-bath random walks with Markov bases" and is authored by Caprice Stanley and Tobias Windisch. It discusses the study of graphs on lattice points and their mixing behavior with heat-bath random walks.'

几个方面简单看一下代码实现原理
* tool
* Agent是怎么执行的
* ZeroShotAgent 的 AgentOutputParser
* 最核心的prompt，ZeroShotAgent

开发一个demo，使用自定义tool，出行助手
* 天气预报/langchain集成
* 机票预定
* 酒店预订

In [19]:
os.environ['openweathermap_api_key'] = "6515bb3582e8427298c0c38d2ada0508"

自定义tool：预定机票，预订酒店


In [17]:
from langchain.tools import StructuredTool

def book_flight(date: str) -> str:
    """userful for when you need to book a flight"""
    return f"book flight for {date} success"

def book_hotel(start_date: str , end_date: str, location: str) -> str:
    """userful for when you need to book a hotel"""
    return f"book hotel in {location} from {start_date} to {end_date} success"

In [21]:
from langchain.agents import AgentType,load_tools,initialize_agent,AgentExecutor,StructuredChatAgent
from langchain.chat_models import ChatOpenAI
import langchain

langchain.verbose = True
llm = ChatOpenAI(api_key=api_key,base_url=api_base)

tools = load_tools(["openweathermap-api"])
tools = tools + [StructuredTool.from_function(book_flight),StructuredTool.from_function(book_hotel)]

agent = initialize_agent(tools,llm,agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

agent.run("我2023-12-15到2023-12-20日要去成都游玩，帮我看看天气如何？然后定好酒店和机票。")



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Respond to the human as helpfully and accurately as possible. You have access to the following tools:

OpenWeatherMap: A wrapper around OpenWeatherMap API. Useful for fetching current weather information for a specified location. Input should be a location string (e.g. London,GB)., args: {{'location': {{'title': 'Location', 'type': 'string'}}}}
book_flight: book_flight(date: str) -> str - userful for when you need to book a flight, args: {{'date': {{'title': 'Date', 'type': 'string'}}}}
book_hotel: book_hotel(start_date: str, end_date: str, location: str) -> str - userful for when you need to book a hotel, args: {{'start_date': {{'title': 'Start Date', 'type': 'string'}}, 'end_date': {{'title': 'End Date', 'type': 'string'}}, 'location': {{'title': 'Location', 'type': 'string'}}}}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (t

'The weather in Chengdu from December 15th to December 20th, 2023 is expected to be clear with temperatures ranging from 10.94°C to 11.91°C. The wind speed will be 3 m/s with a direction of 340° and the humidity will be 71%. There is no rain expected during this period. The hotel in Chengdu has been booked from December 15th to December 20th, 2023 and the flight for December 15th, 2023 has also been booked.'

然后从一些代码细节和提示词整个输入输出流程中，可以了解到langchain的tool和agent的原理是怎样的。
基本上，langchain的agent就是使用了提示词技巧中的react框架，然后加上类似伪代码中的逻辑完成的。

额外多开一起，讲讲现在最热门几个agent/tool开源项目。